In [1]:
import sys
sys.path.append('..')

%load_ext autoreload
%autoreload 2

In [2]:
from datasets.data_module import MIMICDataModule
from omegaconf import OmegaConf

config_path = '../configs/mimic_classification_config.yaml'
cfg = OmegaConf.load(config_path)

In [3]:
train_dataloader = MIMICDataModule(cfg).train_dataloader()

In [5]:
for batch in train_dataloader:
    report = batch['report'][0]
    break

In [6]:
from transformers import (
    TokenClassificationPipeline,
    AutoModelForTokenClassification,
    AutoTokenizer,
)
from transformers.pipelines import AggregationStrategy
import numpy as np

# Define keyphrase extraction pipeline
class KeyphraseExtractionPipeline(TokenClassificationPipeline):
    def __init__(self, model, *args, **kwargs):
        super().__init__(
            model=AutoModelForTokenClassification.from_pretrained(model),
            tokenizer=AutoTokenizer.from_pretrained(model),
            *args,
            **kwargs
        )

    def postprocess(self, all_outputs):
        results = super().postprocess(
            all_outputs=all_outputs,
            aggregation_strategy=AggregationStrategy.FIRST,
        )
        return np.unique([result.get("word").strip() for result in results])


In [7]:
# Load pipeline
model_name = "ml6team/keyphrase-extraction-distilbert-inspec"
extractor = KeyphraseExtractionPipeline(model=model_name)


In [9]:
# report

In [10]:
keyphrases = extractor(report)

print(keyphrases)

['acute cardiopulmonary process' 'bilateral nodular opacities'
 'cardiomediastinal silhouette' 'chronic deformity' 'comparison'
 'final report examination' 'focal consolidation' 'infection technique'
 'lat' 'nipple shadows' 'pleural effusion' 'pneumothorax' 'seventh ribs']
